# Video Summarizer

### Dependencies

In [41]:
import re
from dotenv import load_dotenv
from pytubefix import YouTube
from pytubefix.cli import on_progress
from openai import OpenAI
from IPython.display import Markdown, display
import os

# For local audio transcription
import whisper
import soundfile as sf
import numpy as np
from pydub import AudioSegment

In [6]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-


### Step 1: Download sound from a Youtube Video

In [7]:
class SoundDownloader:
    def __init__(self, url):
        self.url = url

    def call(self):
        yt = YouTube(self.url)
        ys = yt.streams.filter(only_audio=True, file_extension='webm').order_by('abr').first()
        print(yt.title)
        output_filename = self.sanitize_filename(yt.title) + '.webm'
        ys.download(filename=f"output_filename")

    def sanitize_filename(self, title):
        # Remove invalid characters and replace slashes with dashes
        clean_title = re.sub(r'[\\/*?:"<>|]', "", title)
        clean_title = clean_title.replace('/', '-')
        return clean_title

In [22]:
urls = ['https://www.youtube.com/watch?v=CT_79oqYB0c']

In [23]:
for url in urls:
    video_sound_downloader = SoundDownloader(url)
    # video_sound_downloader.call()

### Step 2: Audio -> Transcript (speech to text)

#### With OpenAI

In [36]:
class AudioTranscriber:
    def __init__(self, audio_file_path):
        self.audio_file_path = audio_file_path
        self.client = OpenAI()

    def call(self):
        with open(self.audio_file_path, "rb") as audio_file:
            transcription = self.client.audio.transcriptions.create(
                model="whisper-1", 
                file=audio_file
            )
        
        output_file = self.audio_file_path.rsplit('.', 1)[0] + '_transcription.txt'
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(transcription.text)
            
        return output_file

In [14]:
audio_file_path = 'Ahmed Sylla & Hakim Jemili, complices du rire - C à Vous - 15042024.webm'

In [15]:
audio_transcriber = AudioTranscriber(audio_file_path)
# audio_transcriber.call()

#### With local Whisper

In [45]:
class LocalAudioTranscriber:
    def __init__(self, audio_file_path):
        self.audio_file_path = audio_file_path
        self.model = whisper.load_model("base")

    def call(self):
        result = self.model.transcribe(self.audio_file_path)
        
        output_file = self.audio_file_path.rsplit('.', 1)[0] + '_local_transcription.txt'
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(result['text'])
            
        return output_file

In [46]:
audio_file_path = 'Ahmed Sylla & Hakim Jemili, complices du rire - C à Vous - 15042024.webm'

In [ ]:
local_audio_transcriber = LocalAudioTranscriber(audio_file_path)
local_audio_transcriber.call()

/opt/anaconda3/lib/python3.12/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


### Step 3: Transcript -> Summary

In [17]:
class TextSummarizer:
    def __init__(self, text_file_path):
        self.text_file_path = text_file_path
        self.client = OpenAI()
        self.system_prompt = """
        You are an assistant that analyzes the content of a video transcription
        and provides a summary of the main topics discussed in the transcript.
        Summarize them in 1000 words or less and use Markdown formatting.
        """
        self.user_prompt = """
        You are looking at a transcript containing the following content.
        Please provide a summary of the main topics discussed in the transcript:
        """
        self.user_prompt += open(self.text_file_path, 'r', encoding='utf-8').read()

    def call(self):
        return display(Markdown(self.__summarize__()))
    
    def __prompts__(self):
        return [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": self.user_prompt}
        ]
    
    def __summarize__(self):
        response = self.client.chat.completions.create(
            model = "gpt-4o-mini",
            messages = self.__prompts__()
        )
        print(response.choices[0].message.content)
        return response.choices[0].message.content


In [18]:
transcript_file_path = 'Ahmed Sylla & Hakim Jemili, complices du rire - C à Vous - 15042024_transcription.txt'

In [20]:
text_summarizer = TextSummarizer(transcript_file_path)
print(text_summarizer.call())

# Résumé des principaux thèmes discutés

Dans le programme "C'est à vous", plusieurs sujets ont été abordés autour de la cuisine, du cinéma, et des problématiques d'identité. Voici un résumé des thèmes centraux discutés durant l'émission :

## 1. **Cuisine : Le chou farci et la gastronomie régionale**
- **Chef Julien Perard** a présenté un plat de chou farci, préparé avec soin dans une quantité appropriée de beurre.
- L'accompagnement incluait une salade composée de pousses de moutarde et de fleurs de bourrache. La discussion autour du repas se concentrait sur l'utilisation optimale des ingrédients, comme le tronc de brocoli.

## 2. **Thématiques identitaires dans le cinéma**
- Le film *Ici et là-bas* a été l'un des sujets majeurs, mettant en avant deux hommes d'origines différentes qui traversent des expériences de découvertes et de confrontations avec leurs identités respectives.
- **Ahmed Sylla** et **Hakim Gemmani**, acteurs du film, ont partagé leurs expériences personnelles liées

# Résumé des principaux thèmes discutés

Dans le programme "C'est à vous", plusieurs sujets ont été abordés autour de la cuisine, du cinéma, et des problématiques d'identité. Voici un résumé des thèmes centraux discutés durant l'émission :

## 1. **Cuisine : Le chou farci et la gastronomie régionale**
- **Chef Julien Perard** a présenté un plat de chou farci, préparé avec soin dans une quantité appropriée de beurre.
- L'accompagnement incluait une salade composée de pousses de moutarde et de fleurs de bourrache. La discussion autour du repas se concentrait sur l'utilisation optimale des ingrédients, comme le tronc de brocoli.

## 2. **Thématiques identitaires dans le cinéma**
- Le film *Ici et là-bas* a été l'un des sujets majeurs, mettant en avant deux hommes d'origines différentes qui traversent des expériences de découvertes et de confrontations avec leurs identités respectives.
- **Ahmed Sylla** et **Hakim Gemmani**, acteurs du film, ont partagé leurs expériences personnelles liées aux questions d'identité culturelle et à la perception du racisme en France. Ils ont évoqué comment leur parcours a eu un impact sur leur façon de se représenter à l'écran.

## 3. **Discussions sur le racisme et les stéréotypes**
- Les acteurs ont débattu des réflexes racistes encore présents dans la société, et la façon dont leur film vise à aborder ces questions sans tomber dans le militantisme.
- Ils ont souligné l'importance de l'auto-évaluation et de l'éducation personnelle pour combattre ces stéréotypes.

## 4. **Le parcours des artistes et le soutien à l'autisme**
- **Francis Perrin**, acteur emblématique, a partagé son engagement en tant que père d'un jeune homme autiste, **Louis**, et la campagne de sensibilisation sur l'autisme.
- Leur parcours a révélé les luttes rencontrées pour obtenir de la reconnaissance dans les méthodes de traitement pour autisme et a mis en lumière les difficultés que les familles rencontrent face à des jugements déplacés.

## 5. **La comédie et le divertissement**
- Les invités, notamment **Ahmed** et **Hakim**, ont également discuté de leur carrière humoristique, abordant la capacité de la comédie à évoquer des vérités personnelles et sociétales tout en divertissant.
- Les dynamiques entre les comédiens sur scène, leur amitié et leurs expériences professionnelles ont été également mises en avant.

## 6. **Appel à la tolérance et acceptation de soi**
- Les échanges autour de la façon dont chacun a appris à accepter ses propres racines et ses différences sont restés en toile de fond, positionnant l'importance de la tolérance dans le cadre de la société moderne.

## 7. **Événements à venir : promotions de films et spectacles**
- Plusieurs événements à venir ont été annoncés, incluant la sortie de *Ici et là-bas* au cinéma et des spectacles d'humour, renforçant l'idée de la continuité des carrières des artistes présents.

## 8. **Culture musicale : Bruce Springsteen**
- Un segment a été consacré à la carrière de Bruce Springsteen, avec des réflexions sur sa notoriété actuelle et son impact sur la musique et la culture.

Cet ensemble de discussions reflète une connexion entre la cuisine, l'art et les expériences individuelles, tout en explorant des dimensions sociales pertinentes à notre époque.

None
